## To What Extent Does Weather Impact Percent On Time of Buses?
   This notebook is intended to be used to take MTBA bus data and weather data collected by a NOAA station in boston to determine if weather has a measurable impact on the percent of buses that are on time. The MTBA data set includes for its bus entries if it's reccord is from peak hours or off peak as well as number of busses on time that day for that route and number of total busses that ran on that route that day. The weather data includes a number of attributes not as useful, but does include inches of snow, rain, as well as Average Temperature, Max Temperature, and Min Temperature. Both datasets entries have dates, which can and will be joined on to get the data sets together. After which the data will be reorganized for use by a machine learning algorithm or two. The goal is to find an algorithm that will produce a model with good performance, and then look at the weights of different weather to come to a conclusion on how impactful weather is for percent on time.  

In [34]:
import pandas as pd
import numpy as np

#### These first few cells are for combining the weather and bus data
I ended up joining the two sets on date and then reducing the columns. The bus data had two columns that could be useful for this which were route number and peak or off peak. I initially removed route number but might try to bring it back in. The peak or off peak attribute I figured would be interesting but discovered it was making all of the models perform very poorly. 

In [18]:
bus = pd.read_csv("bus1.csv")
weath = pd.read_csv("weather_trimmed.csv")
bus.tail()


,DATE,PRCP,SNOW,TAVG,TMAX,TMIN
838,4/20/2018,0.0,0.0,44.0,53.0,36.0
839,4/21/2018,0.0,0.0,47.0,59.0,39.0
840,4/22/2018,0.0,0.0,51.0,62.0,42.0
841,4/23/2018,0.0,0.0,51.0,58.0,43.0
842,NaN,NaN,NaN,NaN,NaN,NaN


#### Above reads in the two datasets, already prepared to be joined, below joins data sets and outputs for further trimming and re-ordering

In [21]:
bw = bus.join(weath.set_index('DATE'), on ='SERVICE_DATE') #join datasets on dates
#bw = bw.drop( columns = ['SERVICE_DATE','PEAK_OFFPEAK_IND']) # date no longer needed, may confuse algorithms
bw.tail()
#bw.to_csv("busWithWeather1.csv")

,SERVICE_DATE,PEAK_OFFPEAK_IND,ROUTE_TYPE,OTP_NUMERATOR,OTP_DENOMINATOR,% on time,PRCP,SNOW,TAVG,TMAX,TMIN
202309,3/27/2018,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",KBR,630,728,0.865385,0.0,0.0,35.0,42.0,28.0
202310,3/27/2018,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",OTH,176,220,0.800000,0.0,0.0,35.0,42.0,28.0
202311,3/27/2018,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",OTH,223,310,0.719355,0.0,0.0,35.0,42.0,28.0
202312,3/27/2018,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",OTH,106,184,0.576087,0.0,0.0,35.0,42.0,28.0
202313,3/27/2018,"Peak Service (Weekdays 6:30-9:30AM, 3:30PM-6:3...",OTH,73,92,0.793478,0.0,0.0,35.0,42.0,28.0


In [22]:
bw.to_csv("bw.csv")

#### Read in resulting data that's prepared for learning. Split data into training and test partitions after random re-ordering of entries

In [42]:
learn = pd.read_csv("bw_cleaned.csv")
#learn.info()

learn = learn.sample(frac = 1)
train_x = learn.iloc[0:120001,0:5]
train_y = learn.iloc[0:120001,6]

test_x = learn.iloc[120002:,0:5]
test_y = learn.iloc[120002:,6]



### The above data includes the "peak or off peak hours" attribute, the bottom does not.

In [52]:
learn2 = pd.read_csv("bw_nopeak.csv")

learn = learn.sample(frac = 1 )
train_x = learn.iloc[0:120001,0:4]
train_y = learn.iloc[0:120001,5]

test_x = learn.iloc[120002:,0:4]
test_y = learn.iloc[120002:,5]


In [38]:
from sklearn.neural_network import MLPRegressor as mlp
from sklearn.linear_model import TheilSenRegressor as TSR, SGDRegressor as SGD, LogisticRegression as LR
from sklearn.svm import LinearSVR as LSVR, SVR
from sklearn.metrics import mean_absolute_error as mae, r2_score as r2, explained_variance_score as evs

In [57]:
## FOR MULITLAYER PERCEPTRON NEURAL NETWORK
mlpr = mlp()
mlpr.fit(train_x,train_y)
training_mae = mae(train_y, mlpr.predict(train_x)) #best is 0
training_r2 = r2(train_y, mlpr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, mlpr.predict(train_x)) #best is 1 , lower worse
print("training :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

#after having removed peak/off peak flag from data set training performance has gone up
test_mae = mae(test_y, mlpr.predict(test_x))
test_r2 = r2(test_y, mlpr.predict(test_x))
test_evs = evs(test_y, mlpr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))


training :: mae: 2.435896, r2: 0.959362, evs: 0.960010
testing :: mae: 2.424955, r2: 0.959711, evs: 0.960369


In [58]:
## FOR THEILSENREGRESSOR 
tsr = TSR()
tsr.fit(train_x,train_y)
training_mae = mae(train_y, tsr.predict(train_x)) #best is 0
training_r2 = r2(train_y, tsr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, tsr.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

## Have a positive r2 value for this regressor, so it might be worth tuning

test_mae = mae(test_y, tsr.predict(test_x))
test_r2 = r2(test_y, tsr.predict(test_x))
test_evs = evs(test_y, tsr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 2.371573, r2: 0.958320, evs: 0.958571
testing :: mae: 2.361883, r2: 0.958742, evs: 0.958989


In [59]:
## FOR STOCHASTIC GRADIENT DESCENT
sgd = SGD(max_iter = 1000)
sgd.fit(train_x,train_y)
training_mae = mae(train_y, sgd.predict(train_x)) #best is 0
training_r2 = r2(train_y, sgd.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, sgd.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

#did much worse than mlp. probably not useful
test_mae = mae(test_y, sgd.predict(test_x))
test_r2 = r2(test_y, sgd.predict(test_x))
test_evs = evs(test_y, sgd.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 3.070260, r2: 0.934703, evs: 0.956831
testing :: mae: 3.067188, r2: 0.935070, evs: 0.957273


In [66]:
## FOR LOGISTIC REGRESSION

lr = LR()
lr.fit(train_x,train_y)
training_mae = mae(train_y, lr.predict(train_x)) #best is 0
training_r2 = r2(train_y, lr.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, lr.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

test_mae = mae(test_y, lr.predict(test_x))
test_r2 = r2(test_y, lr.predict(test_x))
test_evs = evs(test_y, lr.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

mae: 5.100791, r2: 0.826813, evs: 0.829544
testing :: mae: 5.101414, r2: 0.826410, evs: 0.829181


In [65]:
## FOR SVM
svm = SVR(kernel = 'rbf',max_iter = 1000)
svm.fit(train_x,train_y)
training_mae = mae(train_y, svm.predict(train_x)) #best is 0
training_r2 = r2(train_y, svm.predict(train_x)) #best 1, can be neg(opposite of helpful good)
training_evs = evs(train_y, svm.predict(train_x)) #best is 1 , lower worse
print("mae: %0.6f, r2: %0.6f, evs: %0.6f" % (training_mae,training_r2,training_evs))

test_mae = mae(test_y, svm.predict(test_x)) ####this one has under performed with the new data, not going to be useful
test_r2 = r2(test_y, svm.predict(test_x))
test_evs = evs(test_y, svm.predict(test_x))
print("testing :: mae: %0.6f, r2: %0.6f, evs: %0.6f" % (test_mae,test_r2,test_evs))

C:\Users\Theo\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


mae: 14.797663, r2: -0.175069, evs: 0.019918
testing :: mae: 14.771081, r2: -0.177553, evs: 0.020015


In [ ]:
## others -> beta regression, nearest neighbors?